In [1]:
import os
import sys

import pandas as pd
import numpy as np

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import train_test_split

C:\Users\seann\AppData\Roaming\Python\Python310\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
X_test_raw = pd.read_csv('data/test_set_features.csv')

X_train_raw = pd.read_csv('data/training_set_features.csv')
y_train_raw = pd.read_csv('data/training_set_labels.csv')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_train_raw, y_train_raw, test_size=0.33)

In [3]:
X_train.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [4]:
y_train

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0
...,...,...,...
26702,26702,0,0
26703,26703,0,0
26704,26704,0,1
26705,26705,0,0


In [ ]:
# important columns: h1n1_concern, h1n1_knowledge, 

In [ ]:
# interger encode: education, income_poverity
# one hot encode: age_group, race, sex
                # marital status, rent_or_own, employment_status
                # hhs_geo_region, census_msa, employment_industry,
                # employment_occupation

In [ ]:
# column transformer
    # integer encode ordinal
    # one hot encode nominal
# build pipeline to predict
# try out diff predictors

In [6]:
X_train.columns

Index(['respondent_id', 'h1n1_concern', 'h1n1_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation'],
      dtype='object')

# Baseline Model

In [ ]:
# try out simple linear classifier using the cols 
# that are already transformed

# 'h1n1_concern', 'h1n1_knowledge',
# 'behavioral_antiviral_meds', 'behavioral_avoidance',
# 'behavioral_face_mask', 'behavioral_wash_hands',
# 'behavioral_large_gatherings', 'behavioral_outside_home',
# 'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
# 'chronic_med_condition', 'child_under_6_months', 'health_worker',
# 'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
# 'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
# 'opinion_seas_risk', 'opinion_seas_sick_from_vacc'


In [ ]:
# drop na, join on id, 

In [8]:
# get wanted cols and drop na
X_base_train_cols = X_train.iloc[:, 0:22]
X_base_train_cols = X_base_train_cols.dropna()
X_base_test_cols = X_test.iloc[:,0:22].dropna()

# merge
full_dropped_train = X_base_train_cols.merge(y_train, how='left', on='respondent_id')
X_base_train = full_dropped_train.iloc[:, 1:-2]
y_base_train = full_dropped_train.iloc[:, -2]

full_dropped_test = X_base_test_cols.merge(y_test, how='left', on='respondent_id')
X_base_test = full_dropped_test.iloc[:, 1:-2] # get all cols minus respondent_id and target vars
y_base_test = full_dropped_test.iloc[:, -2] # target vars



base_fit = RidgeClassifierCV().fit(X_base_train, y_base_train)

In [9]:
base_preds = base_fit.decision_function(X_base_test)

In [10]:
roc_auc_score(y_base_test, base_preds)

0.8476478940571242

In [11]:
# Find features with correlation

In [21]:
# merge y_train_raw and X_train_raw
raw_merged = X_train_raw.merge(y_train_raw, on='respondent_id')

In [25]:
# get correlations between features and h1n1_vaccine
raw_merged.corr().iloc[:-2,-2].sort_values(ascending=False)

doctor_recc_h1n1               0.393890
opinion_h1n1_risk              0.323265
opinion_h1n1_vacc_effective    0.269347
opinion_seas_risk              0.258571
doctor_recc_seasonal           0.209864
opinion_seas_vacc_effective    0.179272
health_worker                  0.169768
h1n1_concern                   0.121929
health_insurance               0.121170
h1n1_knowledge                 0.117951
chronic_med_condition          0.095207
opinion_h1n1_sick_from_vacc    0.075091
behavioral_wash_hands          0.074712
behavioral_touch_face          0.071648
behavioral_face_mask           0.070498
child_under_6_months           0.066962
behavioral_avoidance           0.047690
behavioral_antiviral_meds      0.040608
behavioral_outside_home        0.021768
behavioral_large_gatherings    0.017822
opinion_seas_sick_from_vacc    0.008360
household_adults               0.007545
respondent_id                 -0.003280
household_children            -0.003320
Name: h1n1_vaccine, dtype: float64

In [26]:
# get correlations between features and seasonal_vaccine
raw_merged.corr().iloc[:-2,-1].sort_values(ascending=False)

opinion_seas_risk              0.390106
doctor_recc_seasonal           0.369190
opinion_seas_vacc_effective    0.361875
opinion_h1n1_risk              0.216625
opinion_h1n1_vacc_effective    0.205072
health_insurance               0.200858
doctor_recc_h1n1               0.198607
chronic_med_condition          0.170174
h1n1_concern                   0.154828
health_worker                  0.127311
behavioral_touch_face          0.120228
h1n1_knowledge                 0.120152
behavioral_wash_hands          0.112414
behavioral_avoidance           0.076395
behavioral_large_gatherings    0.064025
behavioral_outside_home        0.053509
behavioral_face_mask           0.050083
opinion_h1n1_sick_from_vacc    0.027404
child_under_6_months           0.012097
behavioral_antiviral_meds      0.006277
respondent_id                 -0.004652
opinion_seas_sick_from_vacc   -0.061510
household_adults              -0.064840
household_children            -0.114614
Name: seasonal_vaccine, dtype: float64

In [28]:
# get correlations between features and h1n1_vaccine
raw_merged.corr(method='spearman').iloc[:-2,-2].sort_values(ascending=False)

doctor_recc_h1n1               0.393890
opinion_h1n1_risk              0.305840
opinion_h1n1_vacc_effective    0.291602
opinion_seas_risk              0.256199
doctor_recc_seasonal           0.209864
opinion_seas_vacc_effective    0.195961
health_worker                  0.169768
h1n1_concern                   0.122666
health_insurance               0.121170
h1n1_knowledge                 0.120748
chronic_med_condition          0.095207
behavioral_wash_hands          0.074712
behavioral_touch_face          0.071648
behavioral_face_mask           0.070498
child_under_6_months           0.066962
opinion_h1n1_sick_from_vacc    0.059032
behavioral_avoidance           0.047690
behavioral_antiviral_meds      0.040608
behavioral_outside_home        0.021768
behavioral_large_gatherings    0.017822
household_adults               0.013360
household_children            -0.002012
opinion_seas_sick_from_vacc   -0.002808
respondent_id                 -0.003280
Name: h1n1_vaccine, dtype: float64

In [29]:
# get correlations between features and seasonal_vaccine
raw_merged.corr(method='spearman').iloc[:-2,-1].sort_values(ascending=False)

opinion_seas_vacc_effective    0.397874
opinion_seas_risk              0.389286
doctor_recc_seasonal           0.369190
opinion_h1n1_risk              0.219161
opinion_h1n1_vacc_effective    0.201753
health_insurance               0.200858
doctor_recc_h1n1               0.198607
chronic_med_condition          0.170174
h1n1_concern                   0.154467
health_worker                  0.127311
h1n1_knowledge                 0.121633
behavioral_touch_face          0.120228
behavioral_wash_hands          0.112414
behavioral_avoidance           0.076395
behavioral_large_gatherings    0.064025
behavioral_outside_home        0.053509
behavioral_face_mask           0.050083
opinion_h1n1_sick_from_vacc    0.030128
child_under_6_months           0.012097
behavioral_antiviral_meds      0.006277
respondent_id                 -0.004652
household_adults              -0.058912
opinion_seas_sick_from_vacc   -0.075427
household_children            -0.121141
Name: seasonal_vaccine, dtype: float64

In [ ]:
# take the top however many correlated features, test diff combos, and see what produces best results

In [7]:
# todo: find best features with correlation,
        # maybe find best for h1n1 and one for flu
#      imputation for missing cols?